In [6]:
import altair as alt
from pathlib import Path
import json
import pandas as pd

In [24]:
source = pd.read_csv('results/2020_07_28_11_36.csv')

alt.Chart(source).mark_bar().encode(
    x='type:O',
    y='value:Q',
    column=alt.Column('benchmark:O',
                     title="",
                     header=alt.Header(labelAngle=45)),
    color='type:O'
).interactive()


alt.Chart(...)